Natural Language Processing<br>
Data is on fake job postings

In [49]:
# NLTK is our Natural-Language-Took-Kit Idk why not working for me nltk not being read
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet

from sklearn.feature_extraction.text import TfidfVectorizer


# You may need to download these from nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger') # may need to use if nltk is updated or something
stopwords = stopwords.words('english')

[nltk_data] Downloading package stopwords to C:\Users\Cenzo
[nltk_data]     Oh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to C:\Users\Cenzo
[nltk_data]     Oh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Cenzo
[nltk_data]     Oh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Cenzo Oh\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [13]:
# Import pandas for data handling
import pandas as pd

# Libraries for helping us with strings
import string

# Regular Expression Library
import re

# Import our text vectorizers
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Import our classifiers
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier

# Import some ML helper function
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

# Import our metrics to evaluate our model
from sklearn import metrics
from sklearn.metrics import classification_report

# Library for plotting
import matplotlib.pyplot as plt
import seaborn as sns

In [14]:
#load the data and print shape
df = pd.read_csv('fake_job_postings.csv')
print(df.shape) #17880 rows, 18 cols

(17880, 18)


In [15]:
#first five rows
df.head() 

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0


In [16]:
#check for nulls
print(df.isnull().sum())

job_id                     0
title                      0
location                 346
department             11547
salary_range           15012
company_profile         3308
description                1
requirements            2695
benefits                7210
telecommuting              0
has_company_logo           0
has_questions              0
employment_type         3471
required_experience     7050
required_education      8105
industry                4903
function                6455
fraudulent                 0
dtype: int64


In [17]:
#check for dups
df.duplicated().sum()

0

In [18]:
#checking for value counts / class balances
df.industry.value_counts()

Information Technology and Services    1734
Computer Software                      1376
Internet                               1062
Marketing and Advertising               828
Education Management                    822
                                       ... 
Shipbuilding                              1
Sporting Goods                            1
Museums and Institutions                  1
Wine and Spirits                          1
Ranching                                  1
Name: industry, Length: 131, dtype: int64

In [46]:
#we want to clean data to have all lowercase, no punc, no boring words
def lower_case(string):
    return string.lower()

def remove_punc(string):
    string = re.sub(r'[^\w\s]','',string) #regex to remove punc
    return string

def remove_stopwords(string): #rids of boring words
    words = word_tokenize(string) #tokenize splits words into an array of words ['this', 'is']
    valid_words = []
    for word in words:
        if word not in stopwords: #ensure we do not use boring words / stop words
            valid_words.append(word)
    string = ' '.join(valid_words)
    return string

def text_pipeline(input): #create a pipeline that applies all text processing we defined
    input = lower_case(input)
    input = remove_punc(input)
    input = remove_stopwords(input)
    return input

#testing this out on 
# df["description_clean"] = df["description"]
df["description_clean"] = df["description"].astype(str).apply(text_pipeline)

# df['description_clean']

print("ORIGINAL DESC\n:", df['description'][6])
print('#',80)
print("CLEANDED DESC\n:", df['description_clean'][6])
print('#',80)

ORIGINAL DESC
: Your Responsibilities: Manage the English-speaking editorial team and build a team of best-in-class editorsSet up content creation schedules and ensure deadlines are adhered toResearch and write about the latest tech topics and news in relation to the Android ecosystemEnsure that the content on the site is of a consistently high qualityBe the face and voice of #URL_874846adb69d98865d05ec57ce2425d9e363ef71e0c8436e59e86a136a508716#
# 80
CLEANDED DESC
: responsibilities manage englishspeaking editorial team build team bestinclass editorsset content creation schedules ensure deadlines adhered toresearch write latest tech topics news relation android ecosystemensure content site consistently high qualitybe face voice url_874846adb69d98865d05ec57ce2425d9e363ef71e0c8436e59e86a136a508716
# 80


In [55]:
# second param
df["location_clean"] = df["location"].astype(str).apply(text_pipeline)
print("ORIGINAL LOC\n:", df['location'][2])
print("CLEANDED LOC\n:", df['location_clean'][2])


# do .fillna() for the empty locations

ORIGINAL LOC
: US, IA, Wever
# 80
CLEANDED LOC
: us ia wever
# 80


In [47]:
df.head()

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent,description_clean
0,1,Marketing Intern,"US, NY, New York",Marketing,NaN,"We're Food52, and we've created a groundbreaki...","Food52, a fast-growing, James Beard Award-winn...",Experience with content management systems a m...,NaN,0,1,0,Other,Internship,NaN,NaN,Marketing,0,food52 fastgrowing james beard awardwinning on...
1,2,Customer Service - Cloud Video Production,"NZ, , Auckland",Success,NaN,"90 Seconds, the worlds Cloud Video Production ...",Organised - Focused - Vibrant - Awesome!Do you...,What we expect from you:Your key responsibilit...,What you will get from usThrough being part of...,0,1,0,Full-time,Not Applicable,NaN,Marketing and Advertising,Customer Service,0,organised focused vibrant awesomedo passion cu...
2,3,Commissioning Machinery Assistant (CMA),"US, IA, Wever",NaN,NaN,Valor Services provides Workforce Solutions th...,"Our client, located in Houston, is actively se...",Implement pre-commissioning and commissioning ...,NaN,0,1,0,NaN,NaN,NaN,NaN,NaN,0,client located houston actively seeking experi...
3,4,Account Executive - Washington DC,"US, DC, Washington",Sales,NaN,Our passion for improving quality of life thro...,THE COMPANY: ESRI – Environmental Systems Rese...,"EDUCATION: Bachelor’s or Master’s in GIS, busi...",Our culture is anything but corporate—we have ...,0,1,0,Full-time,Mid-Senior level,Bachelor's Degree,Computer Software,Sales,0,company esri environmental systems research in...
4,5,Bill Review Manager,"US, FL, Fort Worth",NaN,NaN,SpotSource Solutions LLC is a Global Human Cap...,JOB TITLE: Itemization Review ManagerLOCATION:...,QUALIFICATIONS:RN license in the State of Texa...,Full Benefits Offered,0,1,1,Full-time,Mid-Senior level,Bachelor's Degree,Hospital & Health Care,Health Care Provider,0,job title itemization review managerlocation f...


In [59]:
X = df["description_clean"].values #shape, predictor
y = df["fraudulent"] #what we are predicting

In [56]:
X

<17880x100395 sparse matrix of type '<class 'numpy.float64'>'
	with 1548370 stored elements in Compressed Sparse Row format>

In [50]:
#using TfidVectorizer
vectorizer = TfidfVectorizer() #initialize our vectorizer

vectorizer.fit(X) #this makes our vocab matrix

X = vectorizer.transform(X) #this transforms your documents into vectors

print(X.shape, type(X)) #18K words, 100K wide. Sparse matrix

(17880, 100395) <class 'scipy.sparse._csr.csr_matrix'>


In [51]:
X[0] #X is now a sparse matrix

<1x100395 sparse matrix of type '<class 'numpy.float64'>'
	with 72 stored elements in Compressed Sparse Row format>

In [52]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42) #split our data into testing training like always

In [54]:
# Initalize our model.
model = MultinomialNB(alpha=.05)


# Fit our model with our training data.
model.fit(X_train, y_train)


# Make new predictions of our testing data. 
y_pred = model.predict(X_test)


# Make predicted probabilites of our testing data
y_pred_proba = model.predict_proba(X_test)

# Evaluate our model
accuracy =  model.score(X_test, y_test)

# Print our evaluation metrics
print("Model Accuracy: %f" % accuracy) #95.8% accuracy

Model Accuracy: 0.958893


In [60]:
# test predicting if job is fraudulent with location  
X = df["location_clean"].values #shape, predictor
y = df["fraudulent"] #what we are predicting
vectorizer = TfidfVectorizer() #initialize our vectorizer

vectorizer.fit(X) #this makes our vocab matrix

X = vectorizer.transform(X) #this transforms your documents into vectors

print(X.shape, type(X)) #18K words, 100K wide. Sparse matrix

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)

# Initalize our model.
model = MultinomialNB(alpha=.05)


# Fit our model with our training data.
model.fit(X_train, y_train)


# Make new predictions of our testing data. 
y_pred = model.predict(X_test)


# Make predicted probabilites of our testing data
y_pred_proba = model.predict_proba(X_test)

# Evaluate our model
accuracy =  model.score(X_test, y_test)

# Print our evaluation metrics
print("Model Accuracy: %f" % accuracy) #94.3%

(17880, 2310) <class 'scipy.sparse._csr.csr_matrix'>
Model Accuracy: 0.943512
